In [1]:
from ela_feature_definition import ela_feature_names
import pandas as pd
import random
import numpy as np

In [45]:
def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [46]:
ela = pd.read_csv('data/aggregated_ela_representation_dropna.csv',index_col=[0,1,2])
ela=ela[set(ela_feature_names).intersection(ela.columns)]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
ela_scaled=scaler.fit_transform(ela)
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,25))
visualizer.fit(ela_scaled)     
visualizer.show()

kmeans = KMeans(n_clusters=visualizer.elbow_value_)
ela['kmeans_cluster']=kmeans.fit_predict(ela_scaled)
ela.groupby('kmeans_cluster').count()


In [47]:
import numpy as np

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [49]:
ela.shape

(221, 63)

In [50]:
from sklearn.cluster import *
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.manifold import TSNE
from sklearn.decomposition import *
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import numpy as np
from pathlib import Path
clustering_results={}
df=ela
for i in range(2, 30):
    results_for_cluster={}
    cluster_name = f'cluster_{i}'
    labels =  AgglomerativeClustering(n_clusters=i, affinity='cosine', linkage='complete').fit(df).labels_
    results_for_cluster['labels'] = labels
    samples_per_cluster={k[0]:v for k,v in pd.DataFrame(labels).value_counts().to_dict().items()}
    results_for_cluster['samples_per_cluster']=samples_per_cluster
    results_for_cluster['silhouette_score'] = metrics.silhouette_score(df, labels, metric='cosine')
    sample_silhouette_values = metrics.silhouette_samples(df, labels, metric='cosine')
    clustering_results[cluster_name]=results_for_cluster

In [51]:
clustering_results_df=pd.DataFrame(clustering_results).T

In [52]:
clustering_results_df

,labels,samples_per_cluster,silhouette_score
cluster_2,"[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 205, 0: 16}",0.962152
cluster_3,"[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: 205, 1: 14, 2: 2}",0.933072
cluster_4,"[3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","{3: 199, 0: 14, 1: 6, 2: 2}",0.907603
cluster_5,"[3, 3, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","{3: 199, 2: 12, 1: 6, 0: 2, 4: 2}",0.905568
cluster_6,"[1, 1, 2, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 199, 2: 12, 0: 6, 4: 2, 3: 1, 5: 1}",0.902641
cluster_7,"[1, 1, 0, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 199, 0: 12, 2: 3, 6: 3, 4: 2, 3: 1, 5: 1}",0.890907
cluster_8,"[1, 1, 2, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 199, 2: 11, 0: 3, 6: 3, 4: 2, 3: 1, 5: 1, ...",0.890371
cluster_9,"[0, 0, 2, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: 199, 2: 11, 6: 3, 1: 2, 4: 2, 3: 1, 5: 1, ...",0.88007
cluster_10,"[4, 1, 2, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 190, 2: 11, 4: 9, 6: 3, 0: 2, 9: 2, 3: 1, ...",0.778966
cluster_11,"[4, 1, 0, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 190, 0: 11, 4: 9, 2: 3, 9: 2, 3: 1, 5: 1, ...",0.771265


In [53]:
clustering_results_df.to_csv('clust.csv')

In [59]:
selected_cluster_counts = [12,14,22]

In [55]:
metric='cosine'
for cluster_count in selected_cluster_counts:
    agglomerative = AgglomerativeClustering(n_clusters=cluster_count,affinity =metric ,linkage='complete')
    labels=agglomerative.fit(ela).labels_
    elements_per_cluster = pd.DataFrame(labels).value_counts()
    print(elements_per_cluster)
    clusters_with_2_or_less_elements=[x[0] for x in elements_per_cluster[elements_per_cluster<=2].index]
    print(clusters_with_2_or_less_elements)
    ela_copy=ela.copy()
    ela_copy[f'clustering_{cluster_count}_clusters']=labels
 
    diverse_functions=ela_copy[ela_copy[f'clustering_{cluster_count}_clusters'].isin(clusters_with_2_or_less_elements)]
    print(diverse_functions[f'clustering_{cluster_count}_clusters'])

0     190
1       9
9       9
2       3
4       2
5       2
3       1
6       1
7       1
8       1
10      1
11      1
dtype: int64
[4, 5, 3, 6, 7, 8, 10, 11]
suite  fid  iid
bbob   1    3       5
            4      11
       14   2       8
       17   2       4
            5      10
       18   5       6
       19   1       5
            4       7
       22   3       4
       23   5       3
Name: clustering_12_clusters, dtype: int64


In [58]:
ela_copy.to_csv('data/hierarchical_clustering_12.csv')